In [119]:
import pandas as pd
import numpy as np
from typing import Tuple

DATA_PATH = "phone_data.csv"
DATE_COL = "date"
DURATION_COL = "duration"
ITEM_COL = "item"
NETWORK_COL = "network"
MONTH_COL = "month"
NETWORK_TYPE_COL = "network_type"

In [95]:
def load_data(path: str) -> pd.DataFrame:
    try:
        if path.lower().endswith((".xls", ".xlsx")):
            df = pd.read_excel(path)
        else:
            df = pd.read_csv(path, dtype=str)
    except Exception as e:
        raise RuntimeError(f"Failed to load data from {path}: {e}")

    df.columns = [c.strip() for c in df.columns]

    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(
            df[DATE_COL], dayfirst=True, errors="coerce", infer_datetime_format=True
        )
    else:
        raise KeyError(f"Expected column '{DATE_COL}' not found.")

    if DURATION_COL in df.columns:
        df[DURATION_COL] = pd.to_numeric(
            df[DURATION_COL].str.replace(",", ".", regex=False), errors="coerce"
        )
    else:
        df[DURATION_COL] = np.nan

    for c in [ITEM_COL, NETWORK_COL, NETWORK_TYPE_COL, MONTH_COL]:
        if c in df.columns:
            df[c] = df[c].fillna("unknown").astype(str)
        else:
            if c == MONTH_COL and DATE_COL in df.columns:
                df[MONTH_COL] = (
                    df[DATE_COL].dt.to_period("M").astype(str).fillna("unknown")
                )
            else:
                df[c] = "unknown"

    return df

df = load_data(DATA_PATH)
df.head()

C:\Users\DK2012 TX\AppData\Local\Temp\ipykernel_22428\1410466242.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[DATE_COL] = pd.to_datetime(
C:\Users\DK2012 TX\AppData\Local\Temp\ipykernel_22428\1410466242.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[DATE_COL] = pd.to_datetime(


,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [96]:
def basic_inspection(df: pd.DataFrame):
    inspection = {
        "head": df.head(),
        "info": df.info(),
        "dtypes": df.dtypes, 
        "non_null_counts": df.count(),
        "describe_numeric": df.select_dtypes(include=[np.number]).describe(),
    }
    return inspection


In [97]:
def unique_values_count(df: pd.DataFrame) -> pd.Series:
    return df.nunique(dropna=True)

unique_values_count(df)

index           830
date            747
duration        221
item              3
month             5
network           9
network_type      6
dtype: int64

In [98]:
def total_calls_count(df: pd.DataFrame) -> int:
    return int((df[ITEM_COL].str.lower() == "call").sum())

def total_call_duration(df: pd.DataFrame) -> float:
    calls = df[df[ITEM_COL].str.lower() == "call"]
    return float(calls[DURATION_COL].sum(skipna=True))

def sms_count(df: pd.DataFrame) -> int:
    return int((df[ITEM_COL].str.lower() == "sms").sum())

print(total_calls_count(df))
print(total_call_duration(df))
print(sms_count(df))

388
92321.0
292


In [99]:
def most_frequent_mobile_network(df: pd.DataFrame) -> str:
    counts = df[NETWORK_COL].value_counts()
    return counts.idxmax() if not counts.empty else None

def average_call_duration(df: pd.DataFrame) -> float:
    calls = df[df[ITEM_COL].str.lower() == "call"]
    return float(calls[DURATION_COL].mean(skipna=True))

def month_with_most_communications(df: pd.DataFrame) -> str:
    if MONTH_COL in df.columns:
        months = df[MONTH_COL].fillna("unknown")
    else:
        months = df[DATE_COL].dt.to_period("M").astype(str)
    return months.value_counts().idxmax()

print(most_frequent_mobile_network(df))
print(average_call_duration(df))
print(month_with_most_communications(df))

Vodafone
237.94072164948454
2014-11


In [100]:
def total_duration_by_network(df: pd.DataFrame) -> pd.Series:
    return df.groupby(NETWORK_COL)[DURATION_COL].sum()

total_duration_by_network(df)

network
Meteor        7233.00
Tesco        13841.00
Three        36551.00
Vodafone     14770.00
data          5164.35
landline     18433.00
special          3.00
voicemail     1775.00
world            7.00
Name: duration, dtype: float64

In [101]:
def day_with_most_calls(df: pd.DataFrame) -> pd.Timestamp:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    calls["day"] = calls[DATE_COL].dt.date
    counts = calls.groupby("day").size()
    if counts.empty:
        return None
    top_day = counts.idxmax()
    return pd.to_datetime(top_day)

day_with_most_calls(df)

Timestamp('2015-01-05 00:00:00')

In [102]:
def usage_percentage(df: pd.DataFrame) -> pd.Series:
    counts = df[ITEM_COL].str.lower().value_counts()
    return (counts / counts.sum()) * 100

usage_percentage(df)

item
call    46.746988
sms     35.180723
data    18.072289
Name: count, dtype: float64

In [103]:
def add_call_type(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    minutes = df[DURATION_COL] / 60.0
    call_type = np.where(
        df[ITEM_COL].str.lower() != "call",
        None,
        np.where(minutes < 1, "short", np.where(minutes <= 5, "medium", "long")),
    ).astype(object)
    df["call_type"] = call_type
    return df

df_with_type = add_call_type(df)
df_with_type.head()

,index,date,duration,item,month,network,network_type,call_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data,None
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile,short
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile,short
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile,short
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile,short


In [104]:
def avg_call_duration_by_network(df: pd.DataFrame) -> pd.Series:
    calls = df[df[ITEM_COL].str.lower() == "call"]
    return calls.groupby(NETWORK_COL)[DURATION_COL].mean().sort_values(ascending=False)

avg_call_duration_by_network(df)

network
landline     438.880952
Three        284.875000
Vodafone     221.530303
Tesco        194.760563
Meteor       133.333333
voicemail     65.740741
Name: duration, dtype: float64

In [105]:
def network_with_highest_avg_call_duration(df: pd.DataFrame) -> str:
    avg = avg_call_duration_by_network(df)
    return avg.idxmax() if not avg.empty else None

network_with_highest_avg_call_duration(df)

'landline'

In [106]:
def daily_communication_count(df: pd.DataFrame) -> pd.Series:
    days = df[DATE_COL].dt.date
    return days.value_counts().sort_index()

daily_communication_count(df)

date
2014-10-15     6
2014-10-16     7
2014-10-17    13
2014-10-18    13
2014-10-19     6
              ..
2015-03-10     1
2015-03-11     1
2015-03-12     1
2015-03-13     3
2015-03-14     2
Name: count, Length: 151, dtype: int64

In [107]:
def daily_total_call_duration(df: pd.DataFrame) -> pd.Series:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    calls["day"] = calls[DATE_COL].dt.date
    return calls.groupby("day")[DURATION_COL].sum()

daily_total_call_duration(df)

day
2014-10-15       48.0
2014-10-16     2854.0
2014-10-17      471.0
2014-10-18     3089.0
2014-10-19      628.0
               ...   
2015-02-28     1305.0
2015-03-01        9.0
2015-03-02      784.0
2015-03-03     3326.0
2015-03-04    10528.0
Name: duration, Length: 118, dtype: float64

In [108]:
def most_active_hour(df: pd.DataFrame) -> int:
    df = df.copy()
    df["hour"] = df[DATE_COL].dt.hour.fillna(0).astype(int)
    return int(df["hour"].value_counts().idxmax())

most_active_hour(df)

6

In [109]:
def item_network_pivot(df: pd.DataFrame) -> pd.DataFrame:
    return df.pivot_table(
        index=NETWORK_COL,
        columns=ITEM_COL,
        values=DURATION_COL,
        aggfunc="count",
        fill_value=0
    )

item_network_pivot(df)

item,call,data,sms
network,,,
Meteor,54,0,33
Tesco,71,0,13
Three,128,0,87
Vodafone,66,0,149
data,0,150,0
landline,42,0,0
special,0,0,3
voicemail,27,0,0
world,0,0,7


In [110]:
def rolling_2day_avg_call_duration(df: pd.DataFrame) -> pd.Series:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    calls = calls.set_index(DATE_COL).sort_index()
    daily_mean = calls[DURATION_COL].resample("D").mean()
    return daily_mean.rolling(2).mean()

rolling_2day_avg_call_duration(df)


date
2014-10-15            NaN
2014-10-16     361.550000
2014-10-17     435.250000
2014-10-18     271.562500
2014-10-19     255.862500
                 ...     
2015-02-28     116.166667
2015-03-01      77.000000
2015-03-02      82.900000
2015-03-03     355.566667
2015-03-04    5541.166667
Freq: D, Name: duration, Length: 141, dtype: float64

In [111]:
def detect_duration_anomalies(df: pd.DataFrame) -> pd.DataFrame:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    mean = calls[DURATION_COL].mean()
    std = calls[DURATION_COL].std()
    anomalies = calls[(calls[DURATION_COL] > mean + 3*std) |
                      (calls[DURATION_COL] < mean - 3*std)]
    return anomalies

detect_duration_anomalies(df)

,index,date,duration,item,month,network,network_type
742,742,2015-02-17 19:09:00,2328.0,call,2015-03,Three,mobile
816,816,2015-03-04 12:29:00,10528.0,call,2015-03,landline,landline


In [112]:
def monthly_summary(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy["month"] = df_copy[DATE_COL].dt.to_period("M").astype(str)
    summary = df_copy.groupby("month").agg(
        count=(ITEM_COL, "count"),
        total_duration=(DURATION_COL, "sum")
    )
    return summary

monthly_summary(df)

,count,total_duration
month,,
2014-10,111,16930.293
2014-11,227,17909.870
2014-12,166,15991.299
2015-01,169,19822.299
2015-02,118,12016.012
2015-03,39,15107.577


In [113]:
def cumulative_call_duration(df: pd.DataFrame) -> pd.Series:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    calls = calls.sort_values(DATE_COL)
    return calls[DURATION_COL].cumsum()

cumulative_call_duration(df)

1         13.0
2         36.0
3         40.0
4         44.0
5         48.0
        ...   
807    79643.0
808    79700.0
809    81025.0
810    81793.0
816    92321.0
Name: duration, Length: 388, dtype: float64

In [114]:
def longest_same_network_sequence(df: pd.DataFrame) -> int:
    calls = df[df[ITEM_COL].str.lower() == "call"].copy()
    if calls.empty:
        return 0
    change_flag = calls[NETWORK_COL] != calls[NETWORK_COL].shift()
    groups = change_flag.cumsum()
    return groups.value_counts().max()

longest_same_network_sequence(df)

np.int64(7)

In [115]:
def time_diff_between_communications(df: pd.DataFrame) -> pd.Series:
    df_sorted = df.sort_values(DATE_COL).copy()
    return df_sorted[DATE_COL].diff()

time_diff_between_communications(df)

0                 NaT
1     0 days 00:00:00
2     0 days 07:48:00
3     0 days 00:02:00
4     0 days 02:39:00
            ...      
825   0 days 17:40:00
826   0 days 00:01:00
827   0 days 06:19:00
828   0 days 17:15:00
829   0 days 00:03:00
Name: date, Length: 830, dtype: timedelta64[ns]

In [116]:
def classify_activity(df: pd.DataFrame) -> pd.Series:
    daily_counts = df[DATE_COL].dt.date.value_counts()
    avg = daily_counts.mean()
    return daily_counts.apply(lambda x: "high activity" if x > avg else "low activity")

classify_activity(df)

date
2014-11-04    high activity
2014-12-30    high activity
2014-11-12    high activity
2015-02-07    high activity
2014-11-26    high activity
                  ...      
2015-03-08     low activity
2015-03-10     low activity
2015-03-09     low activity
2015-03-12     low activity
2015-03-11     low activity
Name: count, Length: 151, dtype: object

In [117]:
def top_network_for_each_item(df: pd.DataFrame) -> pd.Series:
    counts = df.groupby([ITEM_COL, NETWORK_COL]).size()
    return counts.groupby(level=0).idxmax()

top_network_for_each_item(df)

item
call      (call, Three)
data       (data, data)
sms     (sms, Vodafone)
dtype: object

In [118]:
def network_summary(df: pd.DataFrame) -> pd.DataFrame:
    calls = df[df[ITEM_COL].str.lower() == "call"]
    sms = df[df[ITEM_COL].str.lower() == "sms"]

    summary = pd.DataFrame({
        "total_calls": calls.groupby(NETWORK_COL).size(),
        "total_sms": sms.groupby(NETWORK_COL).size(),
        "total_duration": calls.groupby(NETWORK_COL)[DURATION_COL].sum(),
        "avg_duration": calls.groupby(NETWORK_COL)[DURATION_COL].mean()
    })

    return summary.fillna(0)

network_summary(df)

,total_calls,total_sms,total_duration,avg_duration
network,,,,
Meteor,54.0,33.0,7200.0,133.333333
Tesco,71.0,13.0,13828.0,194.760563
Three,128.0,87.0,36464.0,284.875000
Vodafone,66.0,149.0,14621.0,221.530303
landline,42.0,0.0,18433.0,438.880952
special,0.0,3.0,0.0,0.000000
voicemail,27.0,0.0,1775.0,65.740741
world,0.0,7.0,0.0,0.000000
